In [0]:
import sys
import re
import numpy as np
import argparse
import random

def tokenize(sent):
  return [x.strip().lower() for x in re.split('(\W+)?', sent) if x.strip()]

In [3]:
sent = "I love movie!"
tokenize(sent)

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


['i', 'love', 'movie', '!']

In [0]:
def map_to_idx(x, vocab):
  return [vocab[w] if w in vocab else 0 for w in x]
def map_to_txt(x,vocab):
  textify = map_to_idx(x, inverse_map(vocab))
  return ' '.join(textify)
def inverse_map(vocab):
  return {v:k for k, v in vocab.items()}
def inverse_dis2txt(X_inp, Y_inp, vocabx, vocaby, outp=None):
  inv_map_x = inverse_map(vocabx)
  inv_map_y = inverse_map(vocaby)
  if outp:
    for x, y, z in zip(X_inp, Y_inp, outp):
      print(' '.join(map_to_idx(x, inv_map_x)))
      print(' '.join(map_to_idx(y, inv_map_y)))
      print(z)
  else:
    for x, y, in zip(X_inp, Y_inp):
      print(' '.join(map_to_idx(x, inv_map_x)))
      print(' '.join(map_to_idx(y, inv_map_y)))
def create_train_examples(X, Y, yspace, num=-1, balanced=True):
  X_inp = []
  Y_inp = []
  outp = []
  for x, y in zip(X, Y):
    neg_samples = yspace[:]
    neg_samples.remove(y)
    if num == -1:
      pass
    else:
      neg_samples = [i for i in random.sample(neg_samples, num)]
    if not balanced:
      X_inp.append(x)
      Y_inp.append(y)
      outp.append([1.0,0.0])
    for yn in neg_samples:
      if balanced:
        X_inp.append(x)
        Y_inp.append(y)
        outp.append([1.0,0.0])
      X_inp.append(x)
      Y_inp.append(yn)
      outp.append([0.0,1.0])
  return X_inp, Y_inp, outp
def check_layer_output_shape(layer, input_data):
    ndim = len(input_data.shape)
    layer.input = K.placeholder(ndim=ndim)
    layer.set_input_shape(input_data.shape)
    expected_output_shape = layer.output_shape[1:]

    function = K.function([layer.input], [layer.get_output()])
    output = function([input_data])[0]
    print(output.shape,expected_output_shape)
    assert output.shape[1:] == expected_output_shape

def mytest():
    input_data = np.random.random((10, 142, 200))
    Y = np.random.random((10, 142, 200))
    alpha = np.random.random((10, 142))
    print(input_data.shape)
    # layer = Reshape(dims=(2, 3))
    # layer = Lambda(get_H_n, output_shape=(200,))
    # Y = Layer()
    # alpha= Layer()
    # Y.set_input_shape((None,142,200))
    # alpha.set_input_shape((None,142))
    # ll=Merge([Y,alpha],mode='join')
    # layer=Lambda(get_R, output_shape=(200,1))
    # layer.set_previous(ll)
    # print(layer.input)
    # func = K.function([layer.input], [layer.get_output()])
    # layer=Lambda(get_Y, output_shape=(110, 200))
    # check_layer_output_shape(layer, input_data)
    sys.exit(0)

def show_weights(model,node_name,indices=[0]):
    Wr=model.nodes[node_name].get_weights()
    for i in indices:
        print(Wr[i][0:5])


def show_output(model,node_name,input_data_dict):
    lout= K.function([model.inputs[i].input for i in model.input_order],
                     [model.nodes[node_name].get_output(train=False)])
    output= lout([input_data_dict[i] for i in model.input_order])[0]
    print('input', input_data_dict['input'][0][0:10])
    print(node_name, output[0][0:5])
    print('input', input_data_dict['input'][1][0:10])
    print(node_name, output[1][0:5])


def categorize(ll):
    new_y_train = []
    for y in ll:
        if y == 1:
            new_y_train += [[0, 1]]
        else:
            new_y_train += [[1, 0]]
    return np.asarray(new_y_train)

if __name__=="__main__":
    pass


In [41]:
import json
#from myutils import *
from keras.preprocessing.sequence import pad_sequences
from collections import Counter

def get_data(data,vocab):
    for d in data:
      prem=map_to_idx(tokenize(d["sentence1"]),vocab)
      hyp=map_to_idx(tokenize(d["sentence2"]),vocab)
      label=d["gold_label"]
      yield prem, hyp , label

def load_data(train,vocab,labels={'neutral':0,'entailment':1,'contradiction':2}):
    X,Y,Z=[],[],[]
    for l,p,h in zip([i[0] for i in train],[i[1] for i in train],[i[2] for i in train]):
      p=map_to_idx(tokenize(p),vocab)
      h=map_to_idx(tokenize(h),vocab)
      # print 'P:',map_to_txt(p,vocab)
      # print 'H:',map_to_txt(h,vocab)
      # print p+" DELIMITER "+h
      # ph=map_to_idx(tokenize(p+" delimiter "+h),vocab)
      # print 'PH:',map_to_txt(ph,vocab)
      # print 'L:',l
      if l in labels:         # get rid of '-'
        X+=[p]
        Y+=[h]
        Z+=[labels[l]]
    return X,Y,Z


def get_vocab(data):
    vocab=Counter()
    for ex in data:
      tokens=tokenize(ex[0])
      tokens+=tokenize(ex[1])
      vocab.update(tokens)
    lst = ["unk", "delimiter"] + [ x for x, y in vocab.items() if y > 0]
    vocab = dict([ (y,x) for x,y in enumerate(lst) ])
    return vocab

def convert2simple(data,out):
    '''
    get the good stuff out of json into a tsv file
    '''
    for d in data:
      print(out, d["sentence1"]+"\t"+d["sentence2"]+"\t"+d["gold_label"])
    out.close()


if __name__=="__main__":

    train=[l.strip().split('\t') for l in open('snli_1.0_train.txt')][1:20000]
    dev=[l.strip().split('\t') for l in open('snli_1.0_dev.txt')]
    test=[l.strip().split('\t') for l in open('snli_1.0_test.txt')]
    labels={'contradiction':-1,'neutral':0,'entailment':1}
    vocab=get_vocab(train)
    X_train,Y_train,Z_train=load_data(train,vocab)
    print(Z_train[0:10])
    X_dev,Y_dev,Z_dev=load_data(dev,vocab)
    print(len(X_train),len(Y_train))
    print(len(X_dev),X_dev[0])

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


[0, 2, 1, 0, 1, 2, 2, 1, 0, 0]
19977 19977
9842 [27, 64, 66, 29, 68, 8, 1189, 8, 55, 8, 102, 8, 113, 8, 1098, 0, 63]


In [0]:
def encode_onehot(labels):
  classes = set(labels)
  classes_dict = {c: np.identity(len(classes))[i,:] for i,c in enumerate(classes)}
  labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
  return labels_onehot

In [43]:
def encode_data(train_data, vocab):
  train = []
  print(len(train_data)) 
  for data in train_data:
    empty = np.zeros((len(vocab)))
    for num in data:
      empty[int(num)] = 1
    train.append(empty)
  return train
X_train = encode_data(X_train, vocab)
Y_train = encode_data(Y_train, vocab)
Z_train = encode_onehot(Z_train)
X_dev = encode_data(X_dev, vocab)
Y_dev = encode_data(Y_dev, vocab)
Z_dev = encode_onehot(Z_dev)
print(len(X_dev), len(Y_dev))

19977
19977
9842
9842
9842 9842


In [44]:
import scipy.sparse as sp
import torch

def build_graph(X_train, Y_train, Z_train):
  print(type(X_train), type(Y_train))
  #train = X_train.extend(Y_train)
  train = X_train + Y_train
  print(train)
  features = sp.csr_matrix(train, dtype = np.float32)
  labels = Z_train
  labels = np.tile(Z_train, (2,1))
  #idx = {list(train[i]):i for i in range(len(train))}
  #idx_X = [X_train[k] for k in range(X_train)]
  #idx_Y = [Y_train[k] for k in range(Y_train)]
  
  edges = [[i,j] for i,j in zip(range(len(X_train)),range(len(X_train),len(train)))]
  edges = np.tile(edges, (2,1))
  # edges = np.array(edges)
  print(edges.shape[0] / 2)
  edges[int(edges.shape[0] / 2) : edges.shape[0],[0,1]] = edges[int(edges.shape[0] / 2) : edges.shape[0],[1,0]]
  # print(edges[int(edges.shape[0] / 2) : edges.shape[0],:])
  adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),shape=(labels.shape[0], labels.shape[0]), dtype=np.float32)

  adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

  features = normalize(features)
  adj = normalize(adj + sp.eye(adj.shape[0]))

  features = torch.FloatTensor(np.array(features.todense()))
  labels = torch.LongTensor(np.where(labels)[1])
  adj = sparse_mx_to_torch_sparse_tensor(adj)

  return adj, features, labels 
 
adj, features, labels = build_graph(X_train, Y_train, Z_train)
print(adj.size())

<class 'list'> <class 'list'>
[array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., 

In [0]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx


def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


In [49]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

from models import GCN

adj, features, labels, = build_graph(X_train, Y_train, Z_train)
adj_dev, features_dev, labels_dev = build_graph(X_dev, Y_dev, Z_dev)
# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=16,
            nclass=labels.max().item() + 1,
            dropout=0.7)
lr = 0.01
weight_decay = 5e-4
optimizer = optim.Adam(model.parameters(),lr=lr, weight_decay=weight_decay)

# if args.cuda:
#     model.cuda()
#     features = features.cuda()
#     adj = adj.cuda()
#     labels = labels.cuda()
#     idx_train = idx_train.cuda()
#     idx_val = idx_val.cuda()
#     idx_test = idx_test.cuda()


def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output, labels)
    acc_train = accuracy(output, labels)
    loss_train.backward()
    optimizer.step()

    fastmode = False
    if not fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output, labels)
    acc_val = accuracy(output, labels)
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))


def test():
    model.eval()
    output_dev = model(features_dev, adj_dev)
    loss_test = F.nll_loss(output_dev, labels_dev)
    acc_test = accuracy(output_dev, labels_dev)
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))


# Train model
t_total = time.time()
epochs = 200
for epoch in range(epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

<class 'list'> <class 'list'>
[array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., 